In [ ]:
%pip install ultralytics pyyaml -q

In [ ]:
# 1. 필요 라이브러리 import
import os
import shutil
from pathlib import Path
import yaml


In [ ]:
# 원본 경로 및 클래스 설정 
# 원본 경로의 각 폴더 이름이 모두 train, val, test, images, labels로 이루어져야함. 만약 아니라면 폴더명과 data.yaml 변경 필수
# 클래스 순서는 무조건 헬멧, 장갑, 안전화 순서로 하기
datasets = [
    {'path': 'PPE-1', 'classes': [0, 3, 2]},
    {'path': 'PPE-2', 'classes': [0, 1, 3]}
]


In [ ]:
# 3. 원하는 클래스 이미지/라벨 데이터 뽑아내는 함수
def extract_classes(original_path, wanted_class_ids, temp_folder='temp_dataset'):
    dataset_name = Path(original_path).name
    
    for split in ['train', 'val', 'test']:
        # 경로 패턴 1: images/train, labels/train
        img_dir1 = os.path.join(original_path, 'images', split)
        lbl_dir1 = os.path.join(original_path, 'labels', split)
        
        # 경로 패턴 2: train/images, train/labels
        img_dir2 = os.path.join(original_path, split, 'images')
        lbl_dir2 = os.path.join(original_path, split, 'labels')
        
        if os.path.exists(img_dir1) and os.path.exists(lbl_dir1):
            img_dir, lbl_dir = img_dir1, lbl_dir1
        elif os.path.exists(img_dir2):
            img_dir = img_dir2
            lbl_dir = lbl_dir2 if os.path.exists(lbl_dir2) else None
        else:
            continue
        
        temp_img_dir = os.path.join(temp_folder, split, 'images')
        temp_lbl_dir = os.path.join(temp_folder, split, 'labels')
        os.makedirs(temp_img_dir, exist_ok=True)
        os.makedirs(temp_lbl_dir, exist_ok=True)
        
        class_mapping = {old_id: new_id for new_id, old_id in enumerate(wanted_class_ids)}
        
        counter = 1
        
        if lbl_dir and os.path.exists(lbl_dir):
            for lbl_file in os.listdir(lbl_dir):
                if not lbl_file.endswith('.txt'):
                    continue
                
                lbl_path = os.path.join(lbl_dir, lbl_file)
                
                with open(lbl_path, 'r') as f:
                    lines = f.readlines()
                
                filtered_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if not parts:
                        continue
                    class_id = int(parts[0])
                    if class_id in class_mapping:
                        parts[0] = str(class_mapping[class_id])
                        filtered_lines.append(' '.join(parts) + '\n')
                
                if not filtered_lines:
                    continue
                
                img_name = os.path.splitext(lbl_file)[0]
                img_found = False
                
                for ext in ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']:
                    img_path = os.path.join(img_dir, img_name + ext)
                    if os.path.exists(img_path):
                        new_img_name = f"{dataset_name}_{split}_{counter:04d}{ext}"
                        new_lbl_name = f"{dataset_name}_{split}_{counter:04d}.txt"
                        
                        shutil.copy2(img_path, os.path.join(temp_img_dir, new_img_name))
                        
                        with open(os.path.join(temp_lbl_dir, new_lbl_name), 'w') as f:
                            f.writelines(filtered_lines)
                        
                        counter += 1
                        img_found = True
                        break
        
        print(f'[{dataset_name}] {split}: {counter-1}개 추출 완료')

# 사용 예시
# extract_classes('/path/to/dataset1', [0, 1, 4])


In [ ]:
# 4. 임시 폴더 데이터를 최종 폴더로 이동
def move_to_final(temp_folder='temp_dataset', final_folder='final_dataset'):
    for split in ['train', 'val', 'test']:
        temp_img = os.path.join(temp_folder, split, 'images')
        temp_lbl = os.path.join(temp_folder, split, 'labels')
        
        final_img = os.path.join(final_folder, split, 'images')
        final_lbl = os.path.join(final_folder, split, 'labels')
        
        os.makedirs(final_img, exist_ok=True)
        os.makedirs(final_lbl, exist_ok=True)
        
        if os.path.exists(temp_img):
            for file in os.listdir(temp_img):
                shutil.move(os.path.join(temp_img, file), os.path.join(final_img, file))
        
        if os.path.exists(temp_lbl):
            for file in os.listdir(temp_lbl):
                shutil.move(os.path.join(temp_lbl, file), os.path.join(final_lbl, file))
    
    shutil.rmtree(temp_folder)
    print(f'최종 폴더 {final_folder}로 이동 완료')

# 사용 예시
# move_to_final()


In [ ]:
# 최종 yaml파일 생성
def create_final_yaml(final_folder='final_dataset'):
    class_names = ['helmet', 'gloves', 'boots']
    
    yaml_data = {
        'path': os.path.abspath(final_folder),
        'train': 'train/images',
        'val': 'val/images',
        'test': 'test/images',
        'nc': len(class_names),
        'names': class_names
    }
    
    yaml_path = os.path.join(final_folder, 'data.yaml')
    
    with open(yaml_path, 'w', encoding='utf-8') as f:
        yaml.dump(yaml_data, f, default_flow_style=False, allow_unicode=True, sort_keys=False)
    
    print(f'YAML 파일 생성 완료: {yaml_path}')
    print(f'클래스: {class_names}')

# 사용 예시
# create_final_yaml()

In [ ]:
# 각 데이터셋에서 원하는 클래스 추출
for dataset in datasets:
    extract_classes(dataset['path'], dataset['classes'])


In [ ]:
# 임시 폴더의 모든 데이터를 최종 폴더로 이동
move_to_final()

In [ ]:
# YAML 파일 생성
create_final_yaml(final_folder='final_dataset')